In [6]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta
import time
import nvda_config1

import twelvedata
from twelvedata import TDClient


In [7]:
# set up login
Cpassword = nvda_config1.mysql_password
td = TDClient(apikey=nvda_config1.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'

# Establish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()

# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000  

In [10]:
# test a time_series request for 1 day. This spot is to verify the previous day is correct.
end_date = datetime.now()
start_date = end_date - timedelta(days=1) 
test_time_series = td.time_series(
        symbol=symbol,
        interval=interval,
        outputsize=outputsize,
        start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"),
    ).with_macd().with_ema().with_rsi()
 # Convert to pandas Dataframes
time_series_df = test_time_series.as_pandas()
print(time_series_df.head(5))


C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\twelvedata\mixins.py:69: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\twelvedata\mixins.py:69: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")
C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\twelvedata\mixins.py:69: FutureWarning: errors='ignore' is deprecated and will raise in a future version. U

                         open    high     low    close   volume     macd  \
datetime                                                                   
2024-07-23 15:59:00  122.4850  122.63  122.37  122.430  1920003 -0.09465   
2024-07-23 15:58:00  122.5601  122.61  122.47  122.490   656363 -0.09316   
2024-07-23 15:57:00  122.4399  122.64  122.40  122.550   660433 -0.09600   
2024-07-23 15:56:00  122.4500  122.48  122.33  122.425   390742 -0.10435   
2024-07-23 15:55:00  122.2500  122.60  122.24  122.440   633110 -0.10075   

                     macd_signal  macd_hist        ema       rsi  
datetime                                                          
2024-07-23 15:59:00     -0.08140   -0.01326  122.49274  40.86860  
2024-07-23 15:58:00     -0.07808   -0.01508  122.50842  43.20629  
2024-07-23 15:57:00     -0.07431   -0.02169  122.51303  45.63022  
2024-07-23 15:56:00     -0.06889   -0.03546  122.50378  39.01178  
2024-07-23 15:55:00     -0.06002   -0.04073  122.52348  39.54820 

C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\twelvedata\mixins.py:69: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


In [14]:
# This section is to test a batch

end_date = datetime.now()
start_date = end_date - timedelta(days=1) 
test_batch = td.time_series(symbol='AAPL,AMD,MSFT,NVDA', interval="1min", outputsize=3, start_date=start_date.strftime("%Y-%m-%d"),
        end_date=end_date.strftime("%Y-%m-%d"))
df = test_batch.with_macd().with_macd(fast_period=10).with_stoch().as_pandas()
print(df.head(10))

                               open       high        low      close   volume  \
AAPL 2024-07-23 15:59:00  224.87000  225.05000  224.87000  225.00999   597610   
     2024-07-23 15:58:00  224.89999  225.00000  224.81000  224.86000   326173   
     2024-07-23 15:57:00  224.81000  224.98950  224.77029  224.89000   210295   
AMD  2024-07-23 15:59:00  153.95000  154.11000  153.91000  154.00999   442028   
     2024-07-23 15:58:00  153.92999  154.02991  153.92000  153.95000   189271   
     2024-07-23 15:57:00  153.95000  154.03000  153.92000  153.92500   126803   
MSFT 2024-07-23 15:59:00  444.45001  444.92001  444.38000  444.81000   314353   
     2024-07-23 15:58:00  444.63000  444.70499  444.38000  444.47000    98341   
     2024-07-23 15:57:00  444.39001  444.76999  444.35001  444.54501    82343   
NVDA 2024-07-23 15:59:00  122.48500  122.63000  122.37000  122.43000  1920003   

                             macd1 macd_signal1 macd_hist1     macd2  \
AAPL 2024-07-23 15:59:00  -0.02337  